In [8]:
# importing packages and libaray
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
import importlib
import Ganmodel
from tensorflow.keras.layers import Conv2D, MaxPooling2D,LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Input

2.1.0


In [9]:
# defining path for accessing dataset
image_dir = 'H:/Downloads/keras_png_slices_data/Dataset/keras_png_slices_seg_train/image'


In [10]:
# function for loading images.
def data_gen(img_folder, batch_size):
    c = 0
    n = os.listdir(img_folder) #List of training images
    np.random.shuffle(n)
    print("Number of Train_images:",len(n))
    
    while (True):
        img = np.zeros((batch_size, 256, 256, 1)).astype('float')
       
        for i in range(c, c+batch_size): #Reading set of images for a mentioned batch-size . 
            train_img = tf.keras.preprocessing.image.img_to_array(tf.keras.preprocessing.image.load_img(
                img_folder+'/'+n[i],color_mode='grayscale', target_size=(256, 256)))/255
            
            img[i-c] = train_img #add to array - img[0], img[1], and so on.
             
        c+=batch_size
        if(c+batch_size>=len(os.listdir(img_folder))):
            c=0
            np.random.shuffle(n)
                  
        return img

In [11]:
# Creating Data-generator to load images
train_gen = data_gen(image_dir, batch_size = 16)
#print("Shape of Train image: ",train_gen[1].shape)

Number of Train_images: 9664
Shape of Train image:  (256, 256, 1)


In [12]:
#Build Generator and Discriminator
generator = Ganmodel.make_generator_model()
discriminator = Ganmodel.make_discriminator_model()

In [14]:
#defining parameter
noise = Input((100,))
num_epochs=15000
batch_size=16
display_interval=1000
losses=[] 

#Defining the Adversarial ground truths 
valid = np.ones((batch_size, 1)) 

#Adding some noise  
valid += 0.05 * np.random.random(valid.shape) 
fake = np.zeros((batch_size, 1)) 
fake += 0.05 * np.random.random(fake.shape)
#print(fake.shape)

#generate image from generator with noise as input
generated_image = generator(noise)

#sending generated image as input for discriminator to discriminate as real or fake
v= discriminator(generated_image)

#combining both models
combined_network = Model(noise, v) 

#compiling combined model with binary_crossentropy loss and Adam optimser with lr=0.0002 and beta=0.5
combined_network.compile(loss='binary_crossentropy', 
                         optimizer=tf.keras.optimizers.Adam(0.0002,0.5)) 